In [1]:
# AiiDA imports.
%load_ext aiida
%aiida
from aiida import common, orm
from surfaces_tools.widgets import series_plotter
import json
import os

In [2]:
from IPython.display import display, clear_output
import utils
import ipywidgets as ipw
from app_widgets import AppWidgets

In [3]:
VIEWERS = {
    "CP2K_AdsorptionE": "view_adsorption_energy.ipynb",
    "CP2K_GeoOpt": "view_geometry_optimization.ipynb",
    "CP2K_CellOpt": "view_geometry_optimization.ipynb",
    "CP2K_ORBITALS": "view_orbitals.ipynb",
    "CP2K_PDOS": "view_pdos.ipynb",
    "CP2K_STM": "view_stm.ipynb",
    "CP2K_AFM": "view_afm.ipynb",
    "CP2K_HRSTM": "view_hrstm.ipynb",
    "CP2K_Phonons": "view_phonons.ipynb",
    "CP2K_NEB": "view_neb.ipynb",
    "CP2K_Replica": "view_replica.ipynb",
    "ReplicaWorkChain": "view_replica.ipynb",
}

def uuids_to_nodesdict(uuids):
    workflows = {}
    nworkflows = 0
    for uuid in uuids:
        try:
            node = orm.load_node(uuid)
            nodeisobsolete = "obsolete" in node.extras and node.extras["obsolete"]
            if node.label in VIEWERS and not nodeisobsolete:
                nworkflows += 1
                if node.label in workflows:
                    workflows[node.label].append(node)
                else:
                    workflows[node.label] = [node]
        except common.NotExistent:
            pass

    return nworkflows, workflows

def get_all_structures_and_geoopts(node):
    """Get all atomistic models that led to the one used in the STM simulation"""
    current_node = node
    all_structures = [node]
    all_geoopts = []
    while current_node is not None:
        if isinstance(current_node, orm.StructureData):
            current_node = current_node.creator
        elif isinstance(current_node, orm.CalcJobNode):
            current_node = current_node.caller
            
        elif isinstance(current_node, orm.WorkChainNode):
            if "GeoOpt" in current_node.label:
                all_geoopts.append(current_node)
                current_node = current_node.inputs.structure
                all_structures.append(current_node)
            else:
                current_node = current_node.caller
    
    return all_structures, all_geoopts

def get_workflows(start_date, end_date):
    qb = orm.QueryBuilder()
    qb.append(
        orm.StructureData,
        filters={
            "extras": {"has_key": "surfaces"},
            "mtime": {"and": [{"<=": end_date}, {">": start_date}]},
        },
    )
    qb.order_by({orm.StructureData: {"mtime": "desc"}})

    # For each structure in QB create a dictionary with info on the workflows computed on it.
    data = []
    for node in qb.all(flat=True):
        # print("node ", node.pk, " extras ", node.extras["surfaces"])
        extras = node.extras["surfaces"]
        nworkflows = 0
        if isinstance(extras, list):
            nworkflows, workflows = uuids_to_nodesdict(node.extras["surfaces"])
        if nworkflows > 0:
            data.append(workflows)
    
    return data

In [4]:
# Widgets
config = utils.read_json("config.json")

# Send simulation to openBIS

In [5]:
# Initialise app widgets
app_widgets = AppWidgets("config.json")
app_widgets.load_dropdown_lists()

## Select experiment and molecule

In [ ]:
display(app_widgets.select_experiment_output)
with app_widgets.select_experiment_output:
    display(app_widgets.experiments_dropdown_boxes, app_widgets.molecule_metadata_boxes)

## Select simulation

In [ ]:
import datetime as datetime
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=20)
data = get_workflows(start_date, end_date)

all_workflows = []
for workflows in data:
    workflows = workflows.values()
    workflows = list(workflows)[0]
    all_workflows.extend(workflows)

workflows_dropdown = utils.Dropdown(description='Workflows', disabled=False, layout = ipw.Layout(width = '993px'), style = {'description_width': "110px"}, options = all_workflows)
display(workflows_dropdown)

In [ ]:
def upload_simulations(b):
    selected_experiment_identifier = app_widgets.experiments_dropdown.value
    
    # Get STM Simulation Workchain from AiiDA
    stm_workchain_node = workflows_dropdown.value
    stm_workchain_pk = stm_workchain_node.pk
    structure_stm = stm_workchain_node.inputs.structure
    structure_stm_pk = structure_stm.pk
    structure_stm_node = load_node(structure_stm_pk)

    # Get Geometry Optimisation Workchain from AiiDA
    all_atom_mods, all_geoopts = get_all_structures_and_geoopts(structure_stm_node)
    all_exist_openbis_atom_mods, all_exist_openbis_geoopts, all_exist_openbis_stms = [], [], []
    
    # Get all simulations and atomistic models from openbis
    openbis_geoopts = app_widgets.openbis_session.get_objects(type = "GEOMETRY_OPTIMISATION")
    openbis_atom_mods = app_widgets.openbis_session.get_objects(type = "ATOMISTIC_MODEL")
    openbis_stms = app_widgets.openbis_session.get_objects(type = "2D_MEASUREMENT")
    
    # Verify which GeoOpts are already in openBIS
    for aiida_geoopt_idx, aiida_geoopt in enumerate(all_geoopts):
        exists_openbis_geoopt = False
        for openbis_geoopt in openbis_geoopts:
            if openbis_geoopt.props.get("wfms-uuid") == aiida_geoopt.uuid:
                all_exist_openbis_geoopts.append([openbis_geoopt, True])
                exists_openbis_geoopt = True
                print(f"GeoOpt {openbis_geoopt.props.get('wfms-uuid')} already exists.")

        if exists_openbis_geoopt == False:
            all_exist_openbis_geoopts.append([None, False])
    
    # Reverse the lists because in openBIS, one should start by building the parents.
    all_geoopts.reverse()
    all_exist_openbis_geoopts.reverse()
    
    # Verify which structures (atomistic models) are already in openBIS
    for aiida_atom_mod_idx, aiida_atom_mod in enumerate(all_atom_mods):
        exists_openbis_atom_mod = False
        for openbis_atom_mod in openbis_atom_mods:
            if openbis_atom_mod.props.get("wfms-uuid") == aiida_atom_mod.uuid:
                all_exist_openbis_atom_mods.append([openbis_atom_mod, True])
                exists_openbis_atom_mod = True
                print(f"Atomistic model {openbis_atom_mod.props.get('wfms-uuid')} already exists.")
            
        if exists_openbis_atom_mod == False:
            all_exist_openbis_atom_mods.append([None, False])
    
    # Reverse the lists because in openBIS, one should start by building the parents.
    all_atom_mods.reverse()
    all_exist_openbis_atom_mods.reverse()
    
    # Verify which simulations are already in openBIS
    exists_openbis_stm = False
    for openbis_stm in openbis_stms:
        if openbis_stm.props.get("wfms-uuid") == stm_workchain_node.uuid:
            all_exist_openbis_stms.append([openbis_stm, True])
            print(f"STM {openbis_stm.props.get('wfms-uuid')} already exists.")
        
    if exists_openbis_stm == False:
        all_exist_openbis_stms.append([None, False])

    # Build atomistic models (structures in AiiDA) in openBIS
    all_atomistic_models = []
    
    for atom_mod_idx, atom_mod in enumerate(all_atom_mods):
        exists_openbis_atom_mod = all_exist_openbis_atom_mods[atom_mod_idx]
        
        if exists_openbis_atom_mod[1] == False:
            # Get Structure details from AiiDA
            atom_mod_ase = atom_mod.get_ase()
            atoms_positions = {'coordinates': {'has_value': [], 'has_unit': 'unit:ANGSTROM'}, 'atoms_symbols': []}
            for i, atom_symbol in enumerate(atom_mod_ase.symbols):
                atoms_positions["coordinates"]["has_value"].append(list(atom_mod_ase.positions[i]))
                atoms_positions['atoms_symbols'].append(atom_symbol)

            # Create Atomistic Model in openBIS
            atomistic_model = app_widgets.openbis_session.new_sample(collection = "/MATERIALS/ATOMISTIC_MODELS/ATOMISTIC_MODEL_COLLECTION", type='ATOMISTIC_MODEL')
            
            atomistic_model.props['$name'] = "Atomistic Model 1"
            atomistic_model.props['wfms_uuid'] = atom_mod.uuid
            # atomistic_model.props['cell_vectors'] = json.dumps({'has_value': atom_mod.cell, 'has_unit': 'unit:ANGSTROM'})
            # atomistic_model.props['periodic_boundary_conditions'] = [int(i) for i in atom_mod_ase.pbc]
            # atomistic_model.props['atoms_positions'] = json.dumps(atoms_positions)
            
            # if len(all_atom_mods) > 1 and atom_mod_idx == len(all_atom_mods) - 1: # If it is the last of more than one structures, it is optimised.
            #     atomistic_model.props['optimised'] = True
            # else:
            #     atomistic_model.props['optimised'] = False
            atomistic_model.save()
            
            all_atomistic_models.append(atomistic_model)
        else:
            all_atomistic_models.append(exists_openbis_atom_mod[0])
    
    # Build GeoOpts in openBIS
    all_geoopts_models = []
    
    for geoopt_index, geoopt in enumerate(all_geoopts):
        geoopt_exist_openbis = all_exist_openbis_geoopts[geoopt_index]
        
        if geoopt_exist_openbis[1] == False:
            parent_atomistic_model = all_atomistic_models[geoopt_index]
            geoopt_model = app_widgets.openbis_session.new_sample(experiment = selected_experiment_identifier, type = 'GEOMETRY_OPTIMISATION', parents = [parent_atomistic_model])
            geoopt_model.props['$name'] = "GeoOpt Simulation"
            geoopt_model.props['wfms_uuid'] = geoopt.uuid
            geoopt_model.save()
            
            # Its plus one because there are N+1 geometries for N GeoOpts
            all_atomistic_models[geoopt_index + 1].add_parents(geoopt_model)
            all_atomistic_models[geoopt_index + 1].save()
            
            all_geoopts_models.append(geoopt_model)
        else:
            all_geoopts_models.append(geoopt_exist_openbis[0])
    
    # Build STM simulation in openBIS
    
    if all_exist_openbis_stms[0][1] == False:
        # Get DFT parameters
        dft_params = dict(stm_workchain_node.inputs.dft_params)
        
        # Simulated Scanning Tunneling Microscopy
        optimised_atomistic_model = all_atomistic_models[-1]
        stm = app_widgets.openbis_session.new_sample(experiment = selected_experiment_identifier, type = '2D_MEASUREMENT', parents = [optimised_atomistic_model])
        stm.props['$name'] = "Simulation STM"
        stm.props['wfms_uuid'] = stm_workchain_node.uuid
        
        # stm_params = dict(stm_workchain_node.inputs.spm_params)
        # stm.props['e_min'] = json.dumps({'has_value': stm_params['--energy_range'][0], 'has_unit': 'unit:EV'})
        # stm.props['e_max'] = json.dumps({'has_value': stm_params['--energy_range'][1], 'has_unit': 'unit:EV'})
        # stm.props['de'] = json.dumps({'has_value': stm_params['--energy_range'][2], 'has_unit': 'unit:EV'})
        
        # stm.props['fwhm'] = json.dumps({'has_value': [float(x) for x in stm_params['--fwhms']], 'has_unit': 'unit:EV'})
        # # stm.props['extrap_plane'] = json.dumps({'has_value': stm_params['--energy_range'][2], 'has_unit': 'unit:ANGSTROM'})
        # stm.props['constant_height'] = json.dumps({'has_value': [float(x) for x in stm_params['--heights']], 'has_unit': 'unit:ANGSTROM'})
        # stm.props['constant_current'] = json.dumps({'has_value': [float(x) for x in stm_params['--isovalues']], 'has_unit': 'isovalue'})
        stm.save()
        
        # stm_simulation_images_zip_filename = series_plotter_inst.create_zip_link_for_openbis()
        
        # stm_simulation_images_dataset = session.new_dataset(
        #     type = "RAW_DATA", 
        #     files = [stm_simulation_images_zip_filename],
        #     sample = stm_simulation_model
        # )
        # stm_simulation_images_dataset.save()

        stm_dataset_filename = "stm_simulation.aiida"
        os.system(f"verdi archive create {stm_dataset_filename} --no-call-calc-backward --no-call-work-backward --no-create-backward -N {stm_workchain_pk}")

        stm_dataset = app_widgets.openbis_session.new_dataset(
            type = "RAW_DATA", 
            files = [stm_dataset_filename],
            sample = stm
        )
        stm_dataset.save()

        # Delete the file after uploading
        os.remove(stm_dataset_filename)

In [ ]:
# add_objects_label = ipw.Label(value = "Objects")
# add_molecule_button = utils.Button(description = "Add molecule")
# add_slab_button = utils.Button(description = "Add slab")
# add_product_button = utils.Button(description = "Add product")
# add_objects_hbox = ipw.HBox([add_objects_label, add_molecule_button, add_slab_button, add_product_button])
# added_objects_accordion = ipw.Accordion()

# def add_molecule_button_action(b):
#     added_objects_accordion_children = list(added_objects_accordion.children)
#     added_objects_accordion.set_title(len(added_objects_accordion_children), "Molecule")
#     added_objects_accordion_children.append(utils.Text(description = "Molecule"))
#     added_objects_accordion.children = added_objects_accordion_children
    
# def add_slab_button_action(b):
#     added_objects_accordion_children = list(added_objects_accordion.children)
#     added_objects_accordion.set_title(len(added_objects_accordion_children), "Slab")
#     added_objects_accordion_children.append(utils.Text(description = "Slab"))
#     added_objects_accordion.children = added_objects_accordion_children

# def add_product_button_action(b):
#     added_objects_accordion_children = list(added_objects_accordion.children)
#     added_objects_accordion.set_title(len(added_objects_accordion_children), "Product")
#     added_objects_accordion_children.append(utils.Text(description = "Product"))
#     added_objects_accordion.children = added_objects_accordion_children

# add_molecule_button.on_click(add_molecule_button_action)
# add_slab_button.on_click(add_slab_button_action)
# add_product_button.on_click(add_product_button_action)

# display(add_objects_hbox)
# display(added_objects_accordion)

Accordion()

In [ ]:
# display(app_widgets.object_dropdown)
display(app_widgets.save_close_buttons_hbox)
app_widgets.create_button.on_click(upload_simulations)
app_widgets.quit_button.on_click(app_widgets.close_notebook)

In [ ]:
# print(dict(x.inputs.spm_params))

{'--dx': '0.15', '--fwhms': ['0.08'], '--heights': ['4.0', '6.0'], '--wfn_file': 'parent_calc_folder/aiida-RESTART.wfn', '--xyz_file': 'parent_calc_folder/aiida.coords.xyz', '--isovalues': ['1e-7'], '--eval_cutoff': '16.0', '--eval_region': ['G', 'G', 'G', 'G', 'n-2.0_C', 'p4.0'], '--output_file': 'stm.npz', '--energy_range': ['-2.00', '2.00', '0.040'], '--hartree_file': 'parent_calc_folder/aiida-HART-v_hartree-1_0.cube', '--p_tip_ratios': 0.0, '--extrap_extent': '5.0', '--basis_set_file': 'parent_calc_folder/BASIS_MOLOPT', '--cp2k_input_file': 'parent_calc_folder/aiida.inp'}
